In [1]:
import pandas as pd
import re

In [3]:
import os
os.chdir('/home/amorales/Documents/projects/bias_2')

In [5]:
# Read signling data
path = 'data/raw/alanin_scanning-cuantitative_values-emax_ec50.csv'
signaling_df = pd.read_csv(path)

# Remove mutations that are not alanin scan
alanin_scan_mask = signaling_df.Mutant.apply(lambda x: bool(re.match('([A-Z])(\d+)$', str(x))))
signaling_df = signaling_df.loc[alanin_scan_mask]

# Remove non usefull columns
cols_of_interst = ['Position','Emax_avg_corr_Gi2_HU210', 'Emax_avg_corr_bArr1_HU210']
signaling_df = signaling_df[cols_of_interst]

# Change column names
signaling_df.columns = ['position', 'gi_emax', 'barr_emax']

# Set mutant number as index
signaling_df = signaling_df.set_index('position')

# Set boolean masks for biased and wt
biased_mask = (signaling_df.barr_emax == 0) & (signaling_df.gi_emax > 0)
wt_mask = (signaling_df.barr_emax > 0) & (signaling_df.gi_emax > 0)
    
# Create a field wih categorical signal information
signaling_df['profile'] = 2
signaling_df.loc[wt_mask, 'profile'] = 0
signaling_df.loc[biased_mask, 'profile'] = 1

# Keep only the categorical labels
signaling_df = signaling_df[['profile']]

# Remove mutants with other signaling profiles
signaling_df = signaling_df[signaling_df.profile != 2]

In [7]:
signaling_df.to_csv('data/interim/signaling_df.csv')